In [197]:
import numpy as np
import pandas as pd
import torch.nn as nn
import h5py
import pickle
import torch
import time
import yaml
import copy
import math
import traceback
import sys
import time
import os
import torch.nn.functional as F
import torchvision
import transformers
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

file_dir = '/home/quang/Documents/XAI_env-main/data/processed/IMS/'
file_name = 'x_train.hdf5'

with h5py.File(file_dir + file_name, 'r') as file:
    dataset1 = file['x_train'][:]  # Replace 'dataset_name' with the actual dataset key or name
    dataset1_shape = dataset1.shape
    a = dataset1.dtype
print(dataset1_shape)

   

(7304, 20)


In [198]:
import h5py

file_dir = '/home/quang/Documents/XAI_env-main/data/processed/IMS/'
file_name = 'y_train.hdf5'

with h5py.File(file_dir + file_name, 'r') as file:
    dataset2 = file['y_train'][:]  # Replace 'dataset_name' with the actual dataset key or name
    dataset2_shape = dataset2.shape

print(dataset2_shape)

(7304, 3)


In [199]:
class CustomDataset:
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        current_sample = torch.tensor(self.data[idx, :], dtype=torch.float)
        current_target = torch.tensor(self.targets[idx], dtype=torch.long)
        data_shape = current_sample.shape[1:]
        return {
            "sample": current_sample,
            "target": current_target,
        }

In [200]:
data = dataset1
data.shape

(7304, 20)

In [201]:
targets = dataset2
targets.shape

(7304, 3)

In [202]:
custom_dataset1 = CustomDataset(data=data,targets=targets)

In [203]:
len(custom_dataset1)

7304

In [204]:
print(custom_dataset1)

In [205]:
custom_dataset1[0]

{'sample': tensor([0.1569, 0.0183, 0.0194, 0.0166, 0.1277, 0.0036, 0.0034, 0.0016, 0.0035,
         0.0034, 0.0028, 0.0036, 0.0047, 0.0036, 0.0046, 0.0066, 0.0068, 0.0120,
         0.0117, 0.0073]),
 'target': tensor([34,  0, 10])}

In [206]:
train_loader = DataLoader(custom_dataset1,
                          batch_size=128,
                          shuffle=True)

In [207]:
for i,data in train_loader:
    print(i)

sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample


In [208]:
for i, data in enumerate(train_loader):
    print(i)
    print(data)

0
{'sample': tensor([[0.0436, 0.0122, 0.0299,  ..., 0.0107, 0.0110, 0.0035],
        [0.1117, 0.0135, 0.0235,  ..., 0.0084, 0.0121, 0.0067],
        [0.0500, 0.0127, 0.0238,  ..., 0.2903, 0.2216, 0.1359],
        ...,
        [0.0859, 0.0160, 0.0245,  ..., 0.0086, 0.0100, 0.0057],
        [0.0914, 0.0166, 0.0243,  ..., 0.0061, 0.0172, 0.0069],
        [0.1049, 0.0185, 0.0253,  ..., 0.0084, 0.0103, 0.0058]]), 'target': tensor([[11,  0, 33],
        [37,  0,  6],
        [ 5,  0,  1],
        [10,  0, 33],
        [32,  0, 11],
        [38,  0,  5],
        [ 5,  0, 38],
        [23,  0, 20],
        [15,  0, 29],
        [35,  0,  9],
        [34,  0, 10],
        [42,  0,  1],
        [40,  0,  4],
        [ 0,  0,  6],
        [ 4,  0, 39],
        [31,  0, 13],
        [31,  0, 12],
        [ 2,  0, 42],
        [32,  0, 12],
        [ 5,  0, 39],
        [13,  0, 30],
        [ 6,  0,  0],
        [ 9,  0, 34],
        [ 4,  0,  2],
        [19,  0, 25],
        [42,  0,  2],
      

{'sample': tensor([[0.0431, 0.0244, 0.0101,  ..., 0.1378, 0.1162, 0.0753],
        [0.1303, 0.0112, 0.0182,  ..., 0.0078, 0.0076, 0.0047],
        [0.1105, 0.0168, 0.0316,  ..., 0.0092, 0.0056, 0.0060],
        ...,
        [0.0475, 0.0111, 0.0220,  ..., 0.0115, 0.0085, 0.0077],
        [0.1559, 0.0187, 0.0228,  ..., 0.0160, 0.0109, 0.0103],
        [0.1183, 0.0158, 0.0296,  ..., 0.0068, 0.0085, 0.0038]]), 'target': tensor([[ 4,  0,  2],
        [18,  0, 26],
        [ 5,  0, 39],
        [ 4,  0,  2],
        [39,  0,  5],
        [ 2,  0, 41],
        [ 6,  0,  0],
        [27,  0, 16],
        [ 7,  0, 36],
        [21,  0, 23],
        [35,  0,  9],
        [17,  0, 27],
        [13,  0, 30],
        [ 6,  0, 38],
        [10,  0, 34],
        [43,  0,  0],
        [ 5,  0, 38],
        [ 0,  0, 44],
        [27,  0, 16],
        [ 3,  0, 41],
        [37,  0,  7],
        [38,  0,  5],
        [26,  0, 18],
        [ 5,  0, 38],
        [34,  0, 10],
        [ 6,  0, 38],
        

{'sample': tensor([[0.0691, 0.0309, 0.0299,  ..., 0.0176, 0.0194, 0.0062],
        [0.1265, 0.0155, 0.0226,  ..., 0.0098, 0.0095, 0.0058],
        [0.0823, 0.0148, 0.0225,  ..., 0.0074, 0.0104, 0.0063],
        ...,
        [0.0403, 0.0251, 0.0168,  ..., 0.0892, 0.0954, 0.0514],
        [0.0528, 0.0141, 0.0203,  ..., 0.0286, 0.0194, 0.0153],
        [0.0604, 0.0082, 0.0367,  ..., 0.0110, 0.0093, 0.0062]]), 'target': tensor([[41,  0,  3],
        [17,  0, 27],
        [18,  0, 25],
        [39,  0,  4],
        [19,  0, 24],
        [35,  0,  8],
        [21,  0, 23],
        [ 1,  0,  5],
        [24,  0, 20],
        [ 3,  0, 41],
        [13,  0, 30],
        [26,  0, 18],
        [12,  0, 31],
        [35,  0,  9],
        [38,  0,  6],
        [ 3,  0, 41],
        [26,  0, 18],
        [ 0,  0,  6],
        [ 0,  0,  6],
        [25,  0, 19],
        [ 6,  0, 38],
        [22,  0, 22],
        [28,  0, 16],
        [ 8,  0, 36],
        [ 5,  0, 39],
        [28,  0, 15],
        

41
{'sample': tensor([[0.0455, 0.0141, 0.0358,  ..., 0.0123, 0.0096, 0.0093],
        [0.1839, 0.0191, 0.0208,  ..., 0.0094, 0.0170, 0.0102],
        [0.0513, 0.0150, 0.0304,  ..., 0.0123, 0.0095, 0.0087],
        ...,
        [0.0926, 0.0110, 0.0225,  ..., 0.0092, 0.0095, 0.0063],
        [0.0703, 0.0121, 0.0304,  ..., 0.0089, 0.0122, 0.0084],
        [0.1593, 0.0183, 0.0357,  ..., 0.0141, 0.0096, 0.0037]]), 'target': tensor([[26,  0, 18],
        [39,  0,  5],
        [27,  0, 17],
        [41,  0,  3],
        [32,  0, 12],
        [21,  0, 23],
        [20,  0, 24],
        [43,  0,  1],
        [ 2,  0,  3],
        [ 3,  0,  3],
        [15,  0, 28],
        [18,  0, 26],
        [38,  0,  6],
        [ 0,  0,  6],
        [23,  0, 20],
        [16,  0, 28],
        [18,  0, 25],
        [13,  0, 31],
        [ 2,  0,  4],
        [44,  0,  0],
        [24,  0, 20],
        [ 2,  0, 42],
        [ 4,  0, 40],
        [ 3,  0, 41],
        [ 0,  0,  6],
        [ 8,  0, 35],
     

55
{'sample': tensor([[0.0751, 0.0178, 0.0282,  ..., 0.0127, 0.0098, 0.0053],
        [0.0733, 0.0098, 0.0196,  ..., 0.0114, 0.0111, 0.0074],
        [0.0346, 0.0094, 0.0255,  ..., 0.0079, 0.0068, 0.0056],
        ...,
        [0.1527, 0.0138, 0.0352,  ..., 0.0099, 0.0132, 0.0058],
        [0.0993, 0.0172, 0.0178,  ..., 0.0155, 0.0209, 0.0107],
        [0.1039, 0.0147, 0.0275,  ..., 0.0070, 0.0071, 0.0039]]), 'target': tensor([[28,  0, 16],
        [21,  0, 22],
        [17,  0, 27],
        [35,  0,  9],
        [32,  0, 11],
        [26,  0, 17],
        [20,  0, 24],
        [ 5,  0, 38],
        [ 1,  0,  5],
        [28,  0, 15],
        [ 7,  0, 37],
        [38,  0,  6],
        [ 1,  0,  5],
        [33,  0, 10],
        [32,  0, 12],
        [ 6,  0,  0],
        [ 5,  0, 39],
        [37,  0,  6],
        [ 3,  0, 41],
        [31,  0, 12],
        [27,  0, 17],
        [27,  0, 17],
        [39,  0,  4],
        [31,  0, 13],
        [ 2,  0,  3],
        [ 9,  0, 35],
     

In [209]:
class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_dim,embed_dim, noise_level):
        super(Autoencoder, self).__init__()
        self.input_size, self.hidden_dim, self.noise_level = input_size, embed_dim,noise_level
        self.embed_dim = embed_dim
        self.fc1 = nn.Linear(self.input_size, self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim, self.input_size)
        
    def encoder(self,x):
        x = self.fc1(x)
        h1 = F.relu(x)
        return h1
    
    def mask(self,x):
        corrupted_x = x + self.noise_level + torch.randn_like(x)   # randn_like  Initializes a tensor where all the elements are sampled from a normal distribution.
        return corrupted_x
    
    def decoder(self, x):
        h2 = self.fc2(x)
        return h2
    
    def forward (self, x):
        out = self.mask(x) # Adding noise to feed the network
        encoder = self.encoder(out)
        decoder = self.decoder(encoder)
        return encoder, decoder 
    
    ## Transformer 
    ### Positional encoding
class PositionalEncoding(nn.Module):
        def __init__(self,d_model, dropout=0.0,max_len=16):
            super(PositionalEncoding, self).__init__()
            pe = torch.zeros(max_len,d_model)
            position = torch.arange(0,max_len, dtype = torch.float).unsqueeze(1)
            
            div_term = torch.exp(torch.arange(0,d_model,2).float()*(-math.log(10000.0) / d_model))
            
            pe[:, 0::2] = torch.sin(position * div_term)
            pe[:, 1::2] = torch.cos(position * div_term)

            pe = pe.unsqueeze(0).transpose(0, 1)

            self.register_buffer('pe', pe)
            
        def forward(self, x):
            x = x + self.pe[:x.size(1), :].squeeze(1)
            return x
        
class Net(nn.Module):
        def __init__(self,feature_size,num_layers,n_head,dropout,noise_level,embed_dim):
            super(Net,self).__init__()
            self.embed_dim = embed_dim
            self.hidden_dim = 4*embed_dim
            self.auto_hidden = int(feature_size / 2)
            input_size = self.auto_hidden
            self.pos = PositionalEncoding(d_model=input_size, max_len=input_size)
            encoder_layers = nn.TransformerEncoderLayer(d_model=input_size, nhead=n_head, dim_feedforward=self.hidden_dim, dropout=dropout)
            self.cell = nn.TransformerEncoder(encoder_layers,num_layers=num_layers)
            self.linear = nn.Linear(input_size,1)
            self.autoencoder = Autoencoder(input_size = feature_size, hidden_dim = self.auto_hidden,embed_dim = embed_dim, noise_level=noise_level)
              
        def forward(self,x):
            batch_size,feature_num, feature_size = x.shape
            encode, decode = self.autoencoder(x.view(batch_size,-1).float()) # Equals batch_size * seq_len
            out = encode.reshape(batch_size,-1,self.auto_hidden)
            out = self.pos(out)
            out = out.reshape(1,batch_size,-1)  #(1,batch_size,feature_size)
            out = self.cell(out)
            out = out.reshape(batch_size,-1)
            out = self.linear(out)
            
            return out, decode
        class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_dim,embed_dim, noise_level):
        super(Autoencoder, self).__init__()
        self.input_size, self.hidden_dim, self.noise_level = input_size, embed_dim,noise_level
        self.embed_dim = embed_dim
        self.fc1 = nn.Linear(self.input_size, self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim, self.input_size)
        
    def encoder(self,x):
        x = self.fc1(x)
        h1 = F.relu(x)
        return h1
    
    def mask(self,x):
        corrupted_x = x + self.noise_level + torch.randn_like(x)   # randn_like  Initializes a tensor where all the elements are sampled from a normal distribution.
        return corrupted_x
    
    def decoder(self, x):
        h2 = self.fc2(x)
        return h2
    
    def forward (self, x):
        out = self.mask(x) # Adding noise to feed the network
        encoder = self.encoder(out)
        decoder = self.decoder(encoder)
        return encoder, decoder 
    
    ## Transformer 
    ### Positional encoding
class PositionalEncoding(nn.Module):
        def __init__(self,d_model, dropout=0.0,max_len=16):
            super(PositionalEncoding, self).__init__()
            pe = torch.zeros(max_len,d_model)
            position = torch.arange(0,max_len, dtype = torch.float).unsqueeze(1)
            
            div_term = torch.exp(torch.arange(0,d_model,2).float()*(-math.log(10000.0) / d_model))
            
            pe[:, 0::2] = torch.sin(position * div_term)
            pe[:, 1::2] = torch.cos(position * div_term)

            pe = pe.unsqueeze(0).transpose(0, 1)

            self.register_buffer('pe', pe)
            
        def forward(self, x):
            x = x + self.pe[:x.size(1), :].squeeze(1)
            return x
        
class Net(nn.Module):
        def __init__(self,feature_size,num_layers,n_head,dropout,noise_level,embed_dim):
            super(Net,self).__init__()
            self.embed_dim = embed_dim
            self.hidden_dim = 4*embed_dim
            self.auto_hidden = int(feature_size / 2)
            input_size = self.auto_hidden
            self.pos = PositionalEncoding(d_model=input_size, max_len=input_size)
            encoder_layers = nn.TransformerEncoderLayer(d_model=input_size, nhead=n_head, dim_feedforward=self.hidden_dim, dropout=dropout)
            self.cell = nn.TransformerEncoder(encoder_layers,num_layers=num_layers)
            self.linear = nn.Linear(input_size,1)
            self.autoencoder = Autoencoder(input_size = feature_size, hidden_dim = self.auto_hidden,embed_dim = embed_dim, noise_level=noise_level)
              
        def forward(self,x):
            batch_size,feature_num, feature_size = x.shape
            encode, decode = self.autoencoder(x.view(batch_size,-1).float()) # Equals batch_size * seq_len
            out = encode.reshape(batch_size,-1,self.auto_hidden)
            out = self.pos(out)
            out = out.reshape(1,batch_size,-1)  #(1,batch_size,feature_size)
            out = self.cell(out)
            out = out.reshape(batch_size,-1)
            out = self.linear(out)
            
            return out, decode
        

In [214]:

model = Net(feature_size = 7304,
            noise_level = 0.01,   
            num_layers=3,
            embed_dim = 16,
            n_head = 4,
            dropout=0.1)

In [211]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=2)
criterion = nn.MSELoss()

In [212]:
print(model)

Net(
  (pos): PositionalEncoding()
  (cell): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=3652, out_features=3652, bias=True)
        )
        (linear1): Linear(in_features=3652, out_features=64, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=64, out_features=3652, bias=True)
        (norm1): LayerNorm((3652,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((3652,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (linear): Linear(in_features=3652, out_features=1, bias=True)
  (autoencoder): Autoencoder(
    (fc1): Linear(in_features=7304, out_features=16, bias=True)
    (fc2): Linear(in_features=16, out_features=7304, bias=True)
  )
)


In [188]:
num_epochs = 100
batch_size = 128
for epoch in range(num_epochs):
    model.train()
    for i,(x,rul) in enumerate(train_loader):
        x = batch["sample"].to(device)
        rul = batch["target"].to(device)
        
        #Forward
        x = x.unsqueeze(1)
        output = model(x)
        loss = criterion(output,rul)
        
        #Backward and Optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print("Epoch{}/{}.Iter{}/{}.Loss{}").format(epoch+1,num_epochs,iter,num_iters,loss)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (8x20 and 7304x16)